In [1]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')


print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4  ../../data/Train/CameraRGB/episode_0003_000261...  episode_0003_000261   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4  ../../data/Train/CameraSeg/episode_0003_000261...  


In [4]:
from models.unet import model_unetVGG16

model = model_unetVGG16(3, image_shape=(320, 416, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 416, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 320, 416, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 416, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 208, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [8]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 16
model_dir = '../../saved_models/unet/unet_v3/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (320, 416))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (320, 416))

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [10]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=67*BATCH_SIZE, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=500,
                   gpus = 1)

Epoch 1/500
67/67 [==============================] - 95s 1s/step - loss: 0.2227 - acc: 0.9245 - val_loss: 0.2472 - val_acc: 0.9249

Epoch 00001: val_loss improved from inf to 0.24721, saving model to ../../saved_models/unet/unet_v3//model.hdf5
Epoch 2/500
67/67 [==============================] - 36s 543ms/step - loss: 0.0686 - acc: 0.9773 - val_loss: 0.1146 - val_acc: 0.9637

Epoch 00002: val_loss improved from 0.24721 to 0.11455, saving model to ../../saved_models/unet/unet_v3//model.hdf5
Epoch 3/500
67/67 [==============================] - 37s 550ms/step - loss: 0.0430 - acc: 0.9851 - val_loss: 0.1663 - val_acc: 0.9638

Epoch 00003: val_loss did not improve
Epoch 4/500
67/67 [==============================] - 37s 551ms/step - loss: 0.0306 - acc: 0.9890 - val_loss: 0.0603 - val_acc: 0.9820

Epoch 00004: val_loss improved from 0.11455 to 0.06035, saving model to ../../saved_models/unet/unet_v3//model.hdf5
Epoch 5/500
67/67 [==============================] - 37s 551ms/step - loss: 0.025


Epoch 00044: val_loss did not improve
Epoch 45/500
67/67 [==============================] - 38s 565ms/step - loss: 0.0098 - acc: 0.9960 - val_loss: 0.0331 - val_acc: 0.9911

Epoch 00045: val_loss did not improve
Epoch 46/500
67/67 [==============================] - 38s 562ms/step - loss: 0.0089 - acc: 0.9962 - val_loss: 0.0356 - val_acc: 0.9915

Epoch 00046: val_loss did not improve
Epoch 47/500
67/67 [==============================] - 38s 565ms/step - loss: 0.0082 - acc: 0.9964 - val_loss: 0.0379 - val_acc: 0.9913

Epoch 00047: val_loss did not improve


In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [11]:
model.load_weights('../../saved_models/unet/unet_v3/model.hdf5')
model.save('../../saved_models/unet/unet_v3/model_saved.h5')